In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from math import pi, cos
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow as tf

# Parámetros iniciales
phase = np.random.rand() * 2 * pi
PtBatMax, PtMaxPanel = 43.68, 24.24
K, Eff = 191.1, 21.528 / 100
TInitial, TFinal, Dt = 0.0, 97.0, 1.0
SocInitial, HardDeck, SoftDeck = 95, 20, 30
NomConsum = 2
VOLTAGE = 12.0

# DataFrame de tareas inicial
TasksDf = pd.DataFrame({
    'TaskID': [1, 2, 3, 4],
    'Power': [150, 2.0, 1.0, 2.5],
    'StartTime': [10, 30, 50, 70],
    'Duration': [5, 1, 5, 1],
    'PriorityT': [0.2, 1, 0.4, 0.3],
    'PriorityE': [0.2, 1, 0.1, 0.9]
})

# Añadir un puntaje de selección basado en la prioridad y la potencia
TasksDf['SelectionScore'] = (TasksDf['PriorityT'] + TasksDf['PriorityE']) / TasksDf['Power']

def available_power(t, phase, k, efficiency, pt_max_panel):
    return max(pt_max_panel * cos(2 * pi * t / TFinal + phase) * efficiency * k, 0)

def execute_tasks(times, tasks_df):
    consumption = np.zeros(len(times))
    for _, task in tasks_df.iterrows():
        start_index = np.argmax(times >= task['StartTime'])
        end_index = np.argmax(times >= task['StartTime'] + task['Duration'])
        if start_index and end_index:
            consumption[start_index:end_index + 1] += task['Power'] / (end_index - start_index + 1)
    return consumption

def manage_soc(times, power, consumption, soc_initial, pt_bat_max, dt, nom_consum):
    soc = np.zeros(len(times))
    soc[0] = soc_initial
    for i in range(1, len(times)):
        energy_generated = power[i - 1] * dt / 60
        energy_consumed = (nom_consum + consumption[i - 1]) * dt / 60
        soc[i] = soc[i - 1] + (energy_generated - energy_consumed) * 100 / pt_bat_max
        soc[i] = max(0, min(100, soc[i]))
    return soc

def calculate_derivatives(data, dt):
    d1 = np.zeros(len(data))
    d2 = np.zeros(len(data))
    for i in range(2, len(data)):
        d1[i] = (3 * data[i] - 4 * data[i - 1] + data[i - 2]) / (2 * dt)
        if i > 2:
            d2[i] = (2 * data[i] - 5 * data[i - 1] + 4 * data[i - 2] - data[i - 3]) / dt**2
    return d1, d2

def sample_parameters(n):
    sampled_params = []
    for _ in range(n):
        k_sampled = norm.rvs(191.1, 5)
        eff_sampled = norm.rvs(0.21528, 0.01)
        sampled_params.append((k_sampled, eff_sampled))
    return sampled_params

def analyze_time_in_soc_ranges(socs, dt):
    ranges = np.arange(0, 110, 10)
    time_in_ranges = {}
    for i in range(len(ranges) - 1):
        range_key = f"{ranges[i]}%-{ranges[i+1]}%"
        time_in_ranges[range_key] = 0

    for soc in socs:
        for i in range(len(ranges) - 1):
            if ranges[i] <= soc < ranges[i+1]:
                range_key = f"{ranges[i]}%-{ranges[i+1]}%"
                time_in_ranges[range_key] += dt
                break

        if soc == 100:
            range_key = f"{ranges[-2]}%-{ranges[-1]}%"
            time_in_ranges[range_key] += dt

    return time_in_ranges

def evaluate_performance(time_in_ranges):
    performance_score = 0
    for range_key, time in time_in_ranges.items():
        if "20%-30%" in range_key:
            performance_score += time * 2
        elif "0%-20%" in range_key:
            performance_score += time * 5
        else:
            performance_score += time
    return performance_score

def adjusted_task_selection(tasks_df, soc, soc_derivs):
    # Ajustar la selección de tareas basada en el SoC y sus derivadas
    if soc < SoftDeck or soc_derivs[-1] < 0:
        tasks_df = tasks_df.sort_values(by='SelectionScore', ascending=False)
    return tasks_df

# Proceso iterativo para simulación y ajuste
iterations = 10  # Define el número de iteraciones para ajustar y evaluar
desired_performance = 90  # Define el umbral de rendimiento deseado

for iteration in range(iterations):
    # Simulación
    times = np.arange(TInitial, TFinal + Dt, Dt)
    powers = [available_power(t, phase, K, Eff, PtMaxPanel) for t in times]
    consumption = execute_tasks(times, TasksDf)
    socs = manage_soc(times, powers, consumption, SocInitial, PtBatMax, Dt, NomConsum)

    # Cálculo de derivadas
    power_derivs, power_derivs2 = calculate_derivatives(powers, Dt)
    soc_derivs, soc_derivs2 = calculate_derivatives(socs, Dt)
    consumption_derivs, consumption_derivs2 = calculate_derivatives(consumption, Dt)

    # Evaluación de rendimiento
    time_in_ranges = analyze_time_in_soc_ranges(socs, Dt)
    performance_score = 100 * evaluate_performance(time_in_ranges) / (TFinal + 1)
    print(f"Iteración {iteration + 1}, Rendimiento: {performance_score:.2f}%")

    if performance_score >= desired_performance:
        print("Rendimiento deseado alcanzado. Finalizando ajustes.")
        break
    else:
        # Ajustar la selección de tareas para la próxima iteración
        TasksDf = adjusted_task_selection(TasksDf, socs[-1], soc_derivs)

# Visualización final del rendimiento y el SoC
plt.figure(figsize=(12, 8))
plt.plot(times, socs, label='SoC', color='green')
plt.xlabel('Tiempo (min)')
plt.ylabel('SoC (%)')
plt.title('Estado de Carga (SoC) a lo Largo del Tiempo')
plt.legend()
plt.grid(True)
plt.show()